In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
from lxml import etree
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, core
import string
from collections import defaultdict
import numpy as np

Using Theano backend.


In [2]:
def xml2df(xml_data):
    tree = ET.parse(xml_data)
    root = tree.getroot()
    all_records = []
    headers = []
    for i, child in enumerate(root):
        record = []
        for subchild in child:
            record.append(subchild.text)
            if subchild.tag not in headers:
                headers.append(subchild.tag)
        all_records.append(record)
    return pd.DataFrame(all_records, columns=headers)

In [3]:
df = xml2df("MathFeedsDataAll.xml")

In [4]:
df.head()
#df.count()

,URL,Title,Text,Domain,blurb,date,image,isbn,kicker,price,timesDeleted,timesEmailed,timesOpened,timesSaved,timesShared,timestamp,wordtitle
0,http://blogs.ams.org/mathgradblog/2017/01/05/d...,Up to Date Blog Content for JMM 2017,Looking for blog content about the 2017 Joint ...,blogs.ams.org,Looking for blog content about the 2017 Joint ...,01/05/17,None,nan,GRADUATE STUDENT BLOG,nan,1.0,nan,4.0,nan,nan,5774489532.046694,uptodateblogcontentforjmm2017
1,http://mrhonner.com/archives/17215,Math Photo: A Dodecagon of Octagons « Mr Honner,I’d never looked closely at the Parachute Jump...,mrhonner.com,I'd never looked closely at the Parachute Jump...,09/18/16,http://MrHonner.com/wp-content/uploads/2016/09...,nan,NOTABLE,nan,nan,nan,nan,nan,nan,5783496448.673001,mathphotoadodecagonofoctagons
2,https://mathbabe.org/2017/03/21/guest-post-the...,Guest post: the age of algorithms | mathbabe,Artie has kindly allowed me to post his though...,mathbabe.org,Artie has kindly allowed me to post his though...,03/21/17,None,nan,BLOG,nan,2.0,2.0,11.0,1.0,nan,5767987618.7198305,guestposttheageofalgorithms
3,http://www.scientificamerican.com/podcast/epis...,Sean M. Carroll Looks at The Big Picture - Sci...,Steve Mirsky: Welcome to Scientific American's...,scientificamerican.com,Caltech theoretical physicist Sean M. Carroll ...,05/12/16,https://www.scientificamerican.com/sciam/cache...,nan,None,nan,nan,nan,nan,nan,nan,5795107200.0,seanmcarrolllooksatthebigpicture
4,http://the-japan-news.com/news/article/0003176002,The Japan News,Not found\n\nThe requested server cannot be ac...,the-japan-news.com,The education ministry will open research cent...,09/13/16,http://the-japan-news.com/modules/img/logo_ogp...,nan,BIG DATA,nan,nan,nan,nan,nan,nan,5784392943.842338,educationministrytopromoteuseofbigdata


In [5]:
text = df['Text'].as_matrix()

In [6]:
doc = text[2]

In [12]:
def clean_word(w):
    strip_str = "()\".?!,"
    new_word = "".join((c for c in w if c in string.printable))
    return new_word.strip(strip_str).lower()

def word_pairs(doc, window=3):
    """
    Returns a list of 2-tuples, which are pairs of words where the second word in the tuple
    appears within 'window' words of the first word.
    
    Parameters
    ----------
    doc (str) : a string representation of the document
    window (int) : how many words to the left and right should be considered 'nearby'
    
    """
    
    words = doc.split()
    word_pairs = []
    for i in range(len(words)):
        word = clean_word(words[i])
        index = i - window
        end = i + window
        while index <= end:
            if index >= 0 and index < len(words) and index != i:
                word_pairs.append((word, clean_word(words[index])))
            index += 1
    return word_pairs

def nearby(doc, window=3):
    """
    Returns a dictionary of every word mapping to a set of the words within 'window' words 
    to either side. This is to say, if window = 3, then the 3 
    
    Parameters
    ----------
    doc (str) : a string representation of the document
    window (int) : how many words to the left and right should be considered 'nearby'
    
    """
    
    words = doc.split()
    nearby_words = defaultdict(set)
    for i in range(len(words)):
        word = clean_word(words[i])
        index = i - window
        end = i + window
        while index <= end:
            if index >= 0 and index < len(words) and index != i:
                nearby_words[word].add(clean_word(words[index]))
            index += 1
    return nearby_words

def vec_of_words(doc):
    """
    Returns a numpy array of every distinct word in the given document.
    
    Parameters
    ----------
    doc (str) : a string representation of the document
    
    """
    
    word_set = set((clean_word(word) for word in doc.split()))
    return np.array(sorted(list(word_set)))

def one_hot(word, all_words):
    """
    Returns a one-hot numpy array of the position of 'word' in 'all_words'.
    
    Parameters
    ----------
    word (str) : the word for which to calculate the one-hot vector
    all_words (numpy array) : vector containing all words in your dictionary
    
    """
    
    w = np.zeros(all_words.shape)
    w[np.where(word_vec == word)] = 1
    return w

def output_vec(word, nearby, all_words):
    """
    Returns the expected output vector for the given word. 
    
    Parameters
    ----------
    word (str) : the word for which to calculate the output vector
    nearby (dict) : dictionary of words mapping to their nearby words
    all_words (numpy array) : vector containing all words in your dictionary
    
    """
    
    output = np.zeros(all_words.shape)
    for w in nearby[word]:
        value = 1 / len(nearby[word])
        output[np.where(word_vec == w)] = value
    return output

def word_vec(word, nearby_pairs, all_words, features=300):
    """
    Returns the word vector constructed by a neural network for the given word.
    
    Parameters
    ----------
    word (str) : the word for which to calculate the output vector
    nearby_pairs (iterable) : contains all pairs where 'word' is the first element
    all_words (numpy array) : vector containing all words in your dictionary
    
    """
    
    model = Sequential()
    model.add(Dense(output_dim=features, input_dim=len(all_words)))
    model.add(Dense(output_dim=len(all_words)))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='sgd')
    
    x = []
    y = []
    word_pairs = [pair for pair in nearby_pairs if pair[0] == word]
    for pair in word_pairs:
        x.append(one_hot(word, all_words).reshape((561,)))
        y.append(one_hot(pair[1], all_words))
    
    x = np.array(x)
    y = np.array(y)
    
    model.fit(x, y, batch_size=1, verbose=0)
    return model

In [15]:
pairs = word_pairs(doc)
pairs

[('artie', 'has'),
 ('artie', 'kindly'),
 ('artie', 'allowed'),
 ('has', 'artie'),
 ('has', 'kindly'),
 ('has', 'allowed'),
 ('has', 'me'),
 ('kindly', 'artie'),
 ('kindly', 'has'),
 ('kindly', 'allowed'),
 ('kindly', 'me'),
 ('kindly', 'to'),
 ('allowed', 'artie'),
 ('allowed', 'has'),
 ('allowed', 'kindly'),
 ('allowed', 'me'),
 ('allowed', 'to'),
 ('allowed', 'post'),
 ('me', 'has'),
 ('me', 'kindly'),
 ('me', 'allowed'),
 ('me', 'to'),
 ('me', 'post'),
 ('me', 'his'),
 ('to', 'kindly'),
 ('to', 'allowed'),
 ('to', 'me'),
 ('to', 'post'),
 ('to', 'his'),
 ('to', 'thoughtful'),
 ('post', 'allowed'),
 ('post', 'me'),
 ('post', 'to'),
 ('post', 'his'),
 ('post', 'thoughtful'),
 ('post', 'email'),
 ('his', 'me'),
 ('his', 'to'),
 ('his', 'post'),
 ('his', 'thoughtful'),
 ('his', 'email'),
 ('his', 'to'),
 ('thoughtful', 'to'),
 ('thoughtful', 'post'),
 ('thoughtful', 'his'),
 ('thoughtful', 'email'),
 ('thoughtful', 'to'),
 ('thoughtful', 'me'),
 ('email', 'post'),
 ('email', 'his'),
 (

In [16]:
vow = vec_of_words(doc)
model = word_vec('algorithm', pairs, vow)